# In Depth A/B Testing - Lab

## Introduction

In this lab, you'll explore a survey from Kaggle regarding budding data scientists. With this, you'll form some initial hypotheses, and test them using the tools you've acquired to date. 

## Objectives

You will be able to:
* Conduct statistical tests on a real-world dataset

## Load the Dataset and Perform a Brief Exploration

The data is stored in a file called **multipleChoiceResponses_cleaned.csv**. Feel free to check out the original dataset referenced at the bottom of this lab, although this cleaned version will undoubtedly be easier to work with. Additionally, meta-data regarding the questions is stored in a file name **schema.csv**. Load in the data itself as a Pandas DataFrame, and take a moment to briefly get acquainted with it.

> Note: If you can't get the file to load properly, try changing the encoding format as in `encoding='latin1'`

In [7]:
#Your code here
import pandas as pd

df = pd.read_csv('multipleChoiceResponses_cleaned.csv', encoding='latin1')


## Wages and Education

You've been asked to determine whether education is impactful to salary. Develop a hypothesis test to compare the salaries of those with Master's degrees to those with Bachelor's degrees. Are the two statistically different according to your results?

> Note: The relevant features are stored in the 'FormalEducation' and 'AdjustedCompensation' features.

In [69]:
#Your code here H1: There is no difference. HA: Salaries of those with Master's degree is higher

Masters = df.loc[df['FormalEducation']=="Master's degree"]
# Masters = Masters.loc[Masters['AdjustedCompensation'].isnull()==False]
Bachelors = df.loc[df['FormalEducation']=="Bachelor's degree"]
# Bachelors = Bachelors.loc[Bachelors['AdjustedCompensation'].isnull()==False]

from scipy.stats import ttest_ind

tstat, pval = ttest_ind(Masters['AdjustedCompensation'], Bachelors['AdjustedCompensation'], equal_var=False, nan_policy='omit')

print(pval/2)
# Since the scipy ttest defaults 2-tail, the p-val is roughly .33, so there is no statistically significant difference


0.3307763945128184


## Wages and Education II

Now perform a similar statistical test comparing the AdjustedCompensation of those with Bachelor's degrees and those with Doctorates. If you haven't already, be sure to explore the distribution of the AdjustedCompensation feature for any anomalies. 

In [70]:
#Your code here H1: There is no difference. HA: Salaries of those with Doctoral degree is higher.

Doctorate = df.loc[df['FormalEducation']=="Doctoral degree"]
# Doctorate = Doctorate.loc[Doctorate['AdjustedCompensation'].isnull()==False]

tstat, pval = ttest_ind(Doctorate['AdjustedCompensation'], Bachelors['AdjustedCompensation'], equal_var=False, nan_policy='omit')

print(pval/2)
# No statistically significant difference with p-val at roughly .15

0.15682381994716163


In [74]:
Doctorate_3above = Doctorate['AdjustedCompensation'].mean() + (Doctorate['AdjustedCompensation'].std()*3)
Doctorate_3below = Doctorate['AdjustedCompensation'].mean() - (Doctorate['AdjustedCompensation'].std()*3)
Doctorate_no_outliers = Doctorate.loc[(Doctorate['AdjustedCompensation'] <= Doctorate_3above) & (Doctorate['AdjustedCompensation'] >= Doctorate_3below)]

Bachelors_3above = Bachelors['AdjustedCompensation'].mean() + (Bachelors['AdjustedCompensation'].std()*3)
Bachelors_3below = Bachelors['AdjustedCompensation'].mean() - (Bachelors['AdjustedCompensation'].std()*3)
Bachelors_no_outliers = Bachelors.loc[(Bachelors['AdjustedCompensation'] <= Bachelors_3above) & (Bachelors['AdjustedCompensation'] >= Bachelors_3below)]

tstat, pval = ttest_ind(Doctorate_no_outliers['AdjustedCompensation'], Bachelors_no_outliers['AdjustedCompensation'], equal_var=False, nan_policy='omit')

print(pval/2)

0.12475225531522234


## Wages and Education III

Remember the multiple comparisons problem; rather than continuing on like this, perform an ANOVA test between the various 'FormalEducation' categories and their relation to 'AdjustedCompensation'.

In [49]:
#Your code here

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

model = ols('AdjustedCompensation ~ FormalEducation', data=df).fit()
table = anova_lm(model, type=2)

table

# With p-val .738, it appears that there is no significant difference in salary across levels of formal education

,df,sum_sq,mean_sq,F,PR(>F)
FormalEducation,6.0,6.540294e+17,1.090049e+17,0.590714,0.738044
Residual,4335.0,7.999414e+20,1.845309e+17,NaN,NaN


## Additional Resources

Here's the original source where the data was taken from:  
    [Kaggle Machine Learning & Data Science Survey 2017](https://www.kaggle.com/kaggle/kaggle-survey-2017)

## Summary

In this lab, you practiced conducting actual hypothesis tests on actual data. From this, you saw how dependent results can be on the initial problem formulation, including preprocessing!